# MLP Classiifers DEF

  - 3. Syndigo Mapping MachineLearning

In [1]:
def unique_no_nan(x):
    return x.dropna().unique()

In [2]:
def constrcut_dic():
    key = unique_no_nan(syndigo_mapped['Level 1'])
    value = list( map(lambda x : re.sub('&|/', '', x), key))
    value = list( map(lambda x : re.sub(' +', '_', x), value))
    value = list(map(str.lower, value))
    #value = ['-'.join(['tfidf'+ str]), value]

    # print(f'key: {key}') 
    # print(f'value: {value}')
    return dict(zip(key, value)) 

In [3]:
def prepare_data_1():
    # Reading PIMMART data
    pim_gtin_mapped = pd.read_csv(DBFR + "PIM_Data_New_50_82Mn.csv", dtype=object)
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD','PMY_DPT_CD', 'REC_DPT_CD', 'ITM_ID', 'GTIN']:
        pim_gtin_mapped[i] = pim_gtin_mapped[i].astype(np.float64)
    
    # Reading Syndigo 259K data
    synd_ALL = pd.read_csv(DBFR + 'Syndigo_Final_ALL.csv', dtype='unicode') # 259k Syndigo Data
    for i in ['SUBCOM_CD', 'DPT_CD', 'COM_CD', 'GTIN', 'ITM_ID', 'PMY_DPT_CD']:
        synd_ALL[i] = synd_ALL[i].astype(np.float64)
    
    # Stripping spaces from all columns
    df_obj = synd_ALL.select_dtypes(['object'])
    synd_ALL[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())
    
    syndigo_mapped = synd_ALL
    pimmart = pim_gtin_mapped
    syndigo_mapped.drop_duplicates('GTIN', inplace = True)
    syndigo_mapped['ITEM_SUBCOM_text'] = \
    (syndigo_mapped.VND_ECOM_DSC + ' ' + syndigo_mapped.SUBCOM_DSC).fillna('').str.lower()
    #syndigo_mapped['Level 1'].value_counts()
    print(f'syndigo_mapped.shape {syndigo_mapped.shape}') 
    #print(f'syndigo_mapped.info() {syndigo_mapped.info()}') 
    #print(f'syndigo_mapped.head() {syndigo_mapped.head()}') 
    #print(f'syndigo_mapped.columns {syndigo_mapped.columns}') 
    
    print(f"syndigo_mapped['Level 1'].value_counts() {syndigo_mapped['Level 1'].value_counts(dropna = False)}")    
    return syndigo_mapped, pim_gtin_mapped

In [68]:
def prepare_data_2(level_1):
    global   k_neighbors
    subset_df = syndigo_mapped[syndigo_mapped['Level 1'] == level_1]

    series =  subset_df['Level 2'].value_counts(ascending=True, dropna = False)
    k_neighbors = 5
    if min(series) < 5:
       
        subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series  < k_neighbors].index.tolist())].index) 
    print (f'Min cnt of Level 2 for class 1  {level_1}  is {min(series)} and k_neighbors is {k_neighbors}')
      
    x_subset_w2v = np.array(list(subset_df.ITEM_SUBCOM_text.apply(lambda x: get_item_vector(x.split(' ')))))                                            
    level2_id_map = dict(zip(subset_df['Level 2'].fillna('Other').unique(), range(subset_df['Level 2'].fillna('Other').nunique())))
    id2_level_map = dict(zip(range(subset_df['Level 2'].fillna('Other').nunique()), subset_df['Level 2'].fillna('Other').unique()))
    y_subset_w2v  = subset_df['Level 2'].fillna('Other').map(level2_id_map)
    
    return  x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map

In [5]:
#Build item vectors
def get_item_vector(item_vocab):
    vect = np.zeros_like(model.get_vector('chips'))
    for word in item_vocab:
        if word in model:
            vect += model.get_vector(word)
    return vect#/max(1,len(item_vocab))

In [6]:
def proc_predict(level_1):

     mlp = MLPClassifier(hidden_layer_sizes=(200,), activation = 'relu', learning_rate= 'adaptive', early_stopping= True)
     #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
     print(f"Training - w2v + MLP \"{level_1}\"")
     warnings.filterwarnings("ignore", category=UserWarning)
     mlp.fit(X_train, y_train)
     print(' Display MLP test  metrics')
     preds_test_mlp = mlp.predict(X_test)
     preds_train_mlp = mlp.predict(X_train)
    
     print(classification_report(y_test, preds_test_mlp,labels = mlp.classes_, target_names = [id2_level_map[i] for i in mlp.classes_]))      
     return 

In [7]:
def get_smp_strgy(dic_y_train, adj_ratio): 
    """
        get_smp_strgy  :  get sampling_strategy for SMOTE
    """
    smp_strgy = {} 
    
    adj_cnt =  int(max(dic_y_train.values()) * adj_ratio)
    for key, values in dic_y_train.items():
        if values < adj_cnt:
           smp_strgy[key] = adj_cnt
        else: 
            smp_strgy [key] = values
            
    return smp_strgy

In [42]:
def proc_smote(): 
    X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= smp_strgy, k_neighbors =   k_neighbors).fit_resample(X_train, y_train)
                                                                                             
    print('Before Resampling:\n',pd.Series(y_train).value_counts())
    print('After Resampling:\n',pd.Series(y_resampled).value_counts())
    mlp_smote = MLPClassifier(hidden_layer_sizes=(200,), activation = 'relu', learning_rate= 'adaptive', early_stopping= True)
    #print(f"Test size: {len(A_test)}\nTrain size: {len(A_train)}\n")
    print(f"Training - w2v + MLP \"{level_1}\"")
    warnings.filterwarnings("ignore", category=UserWarning)
    mlp_smote.fit(X_resampled, y_resampled)
    
    print(' Display MLP + SMOTE test  metrics')
    preds_test_mlp_smote = mlp_smote.predict(X_test)
    preds_train_mlp_smote = mlp_smote.predict(X_resampled)
    print(classification_report(y_test, preds_test_mlp_smote,labels = mlp_smote.classes_, target_names = [id2_level_map[i] for i in mlp_smote.classes_]))
    return

In [49]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier,  RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import gensim
from imblearn.over_sampling import SMOTE, ADASYN
import logging

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
import re

In [10]:
DBFS = "/dbfs/FileStore/tables/OFFSHORE/"
DBFO = "/dbfs/FileStore/tables/OFFSHORE/"
DBFM = "/dbfs/FileStore/tables/MALLIK/"
DBFR = "/dbfs/FileStore/tables/OFFSHORE_RESULTS/"
path = 'C:\\users\\iny2819\\kroger\\Data\\'   
DBFS = path
DBFO = path
DBFM = path
DBFR = path


In [11]:
syndigo_mapped, pim_gtin_mapped = prepare_data_1()

syndigo_mapped.shape (259085, 24)
syndigo_mapped['Level 1'].value_counts() Food / Beverages                            131593
Health & Beauty                              62259
Beer / Wine / Spirits                        17992
Cleaning & Janitorial                         9084
Livestock & Pet Supplies                      6325
Kitchen & Bathroom                            5268
Home & Venue Decoration                       3924
Toys / Games / Hobbies                        3191
Gardening & Outdoors                          2208
Childcare                                     2111
Office Supplies                               1671
Electronics                                   1499
Apparel                                       1283
Lighting & Fans                               1252
Tobacco Products                              1252
NaN                                           1132
Not classified                                1018
Hardware                                       923
Arts & 

In [12]:
dic_level1= constrcut_dic()


In [13]:
stop

NameError: name 'stop' is not defined

In [66]:
#dic_test={"Hospitality Supplies":"hospitality_spplies", "Tobacco Products":"bacco_products"}

### import word2vec model trained on entire PIMMART data
from gensim.models import Word2Vec, KeyedVectors
#model = KeyedVectors.load_word2vec_format('/dbfs/FileStore/tables/DATA_SCIENCE/w2vmodel_053123_PIM_ALL.bin', binary=True)
model = KeyedVectors.load_word2vec_format(path + 'w2vmodel_053123_PIM_ALL.bin', binary=True)                                        
model.most_similar('chips')
adj_ratio = 0.2
i =0
err_cnt = 0
#N = 38 
dic_level1= constrcut_dic()
#dic_level1 = dict(list(dic_level1.items())[4: N])

for level_1, filenamee in dic_level1.items():
    i += 1
    print(f'{i}. Process level 1 = {level_1}')
    x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map =  prepare_data_2(level_1)
    
    X_train, X_test, y_train, y_test = train_test_split(x_subset_w2v, y_subset_w2v.values, test_size= 0.2, stratify=y_subset_w2v, random_state=42)

    proc_predict(level_1)
    dic_y_train = dict(pd.Series(y_train).value_counts())
    smp_strgy = get_smp_strgy(dic_y_train, adj_ratio)
   
    try:
       proc_smote()
    except Exception as e:
       err_cnt += 1 
       logging.error("Exception occurred", exc_info=True)
    

1. Process level 1 = Food / Beverages
Min cnt of Level 2 for class 1  Food / Beverages  is 309 and k_neighbors is 5
Training - w2v + MLP "Food / Beverages"
 Display MLP test  metrics
                                                          precision    recall  f1-score   support

                                    Dairy & Egg Products       0.91      0.92      0.91      1804
                                                 Grocery       0.93      0.96      0.95     14973
                                               Beverages       0.96      0.95      0.96      4248
Meat / Poultry / Seafood / Meat Substitutes (Perishable)       0.74      0.67      0.70       795
                                            Frozen Foods       0.89      0.88      0.89      2716
                                           Bakery / Deli       0.58      0.50      0.54      1067
                                     Baby & Toddler Food       0.86      0.89      0.87       324
                                

Training - w2v + MLP "Cleaning & Janitorial"


ERROR:root:Exception occurred
Traceback (most recent call last):
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_17620\399226564.py", line 25, in <module>
    proc_smote()
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_17620\3537099846.py", line 2, in proc_smote
    X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= smp_strgy, k_neighbors =   k_neighbors).fit_resample(X_train, y_train)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 203, in fit_resample
    return super().fit_resample(X, y)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 88, in fit_resample
    output = self._fit_resample(X, y)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\over_sampling\_smote\base.py", line 355, in _fit_resample
    nns = self.nn_k_.kneighbors(X_class, return_distance=False)[:, 1:]
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\sklearn\

 Display MLP test  metrics
                                   precision    recall  f1-score   support

                   Air Fresheners       0.95      0.97      0.96       354
               Cleaning Chemicals       0.97      0.96      0.97       897
                   Paper Products       1.00      0.98      0.99       187
           General Cleaning Tools       0.86      0.89      0.88       190
           Storage & Organization       0.86      0.86      0.86        81
                Trash & Recycling       0.97      0.94      0.95        77
               Product Dispensers       0.67      0.44      0.53         9
              Carpet & Floor Care       0.00      0.00      0.00         4
      Moisture & Mildew Absorbers       1.00      0.60      0.75         5
       Vacuum Parts & Accessories       0.80      0.67      0.73         6
                         Sweepers       0.00      0.00      0.00         1
                       Absorbents       0.00      0.00      0.00        

ERROR:root:Exception occurred
Traceback (most recent call last):
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_17620\399226564.py", line 25, in <module>
    proc_smote()
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_17620\3537099846.py", line 2, in proc_smote
    X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= smp_strgy, k_neighbors =   k_neighbors).fit_resample(X_train, y_train)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 203, in fit_resample
    return super().fit_resample(X, y)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 88, in fit_resample
    output = self._fit_resample(X, y)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\over_sampling\_smote\base.py", line 355, in _fit_resample
    nns = self.nn_k_.kneighbors(X_class, return_distance=False)[:, 1:]
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\sklearn\

...

ERROR:root:Exception occurred
Traceback (most recent call last):
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_17620\399226564.py", line 25, in <module>
    proc_smote()
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_17620\3537099846.py", line 2, in proc_smote
    X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= smp_strgy, k_neighbors =   k_neighbors).fit_resample(X_train, y_train)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 203, in fit_resample
    return super().fit_resample(X, y)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 88, in fit_resample
    output = self._fit_resample(X, y)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\over_sampling\_smote\base.py", line 355, in _fit_resample
    nns = self.nn_k_.kneighbors(X_class, return_distance=False)[:, 1:]
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\sklearn\

Training - w2v + MLP "Heating / Ventilation / Air Conditioning"
 Display MLP test  metrics
               precision    recall  f1-score   support

Air Treatment       1.00      0.87      0.93        23
      Heating       0.84      1.00      0.91        16

     accuracy                           0.92        39
    macro avg       0.92      0.93      0.92        39
 weighted avg       0.94      0.92      0.92        39

Before Resampling:
 0    88
1    65
dtype: int64
After Resampling:
 0    88
1    65
dtype: int64
Training - w2v + MLP "Heating / Ventilation / Air Conditioning"
 Display MLP + SMOTE test  metrics
               precision    recall  f1-score   support

Air Treatment       1.00      0.91      0.95        23
      Heating       0.89      1.00      0.94        16

     accuracy                           0.95        39
    macro avg       0.94      0.96      0.95        39
 weighted avg       0.95      0.95      0.95        39

27. Process level 1 = Hospitality Supplies
Min 

ERROR:root:Exception occurred
Traceback (most recent call last):
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_17620\399226564.py", line 25, in <module>
    proc_smote()
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_17620\3537099846.py", line 2, in proc_smote
    X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= smp_strgy, k_neighbors =   k_neighbors).fit_resample(X_train, y_train)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 203, in fit_resample
    return super().fit_resample(X, y)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 88, in fit_resample
    output = self._fit_resample(X, y)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\over_sampling\_smote\base.py", line 355, in _fit_resample
    nns = self.nn_k_.kneighbors(X_class, return_distance=False)[:, 1:]
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\sklearn\

 Display MLP test  metrics
                               precision    recall  f1-score   support

                      Apparel       1.00      0.86      0.92         7
Universal Parts & Accessories       0.50      1.00      0.67         1

                     accuracy                           0.88         8
                    macro avg       0.75      0.93      0.79         8
                 weighted avg       0.94      0.88      0.89         8

30. Process level 1 = Power Transmission & Motion Control
Min cnt of Level 2 for class 1  Power Transmission & Motion Control  is 1 and k_neighbors is 3


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [67]:
err_cnt

12

## wip

In [48]:
import logging
try:
       proc_smote()
except Exception as e:
       logging.error("Exception occurred", exc_info=True)
print("test")

ERROR:root:Exception occurred
Traceback (most recent call last):
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_17620\2635623801.py", line 3, in <module>
    proc_smote()
  File "C:\Users\iny2819\AppData\Local\Temp\ipykernel_17620\3537099846.py", line 2, in proc_smote
    X_resampled, y_resampled = SMOTE(random_state= 42, sampling_strategy= smp_strgy, k_neighbors =   k_neighbors).fit_resample(X_train, y_train)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 203, in fit_resample
    return super().fit_resample(X, y)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\base.py", line 88, in fit_resample
    output = self._fit_resample(X, y)
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\imblearn\over_sampling\_smote\base.py", line 355, in _fit_resample
    nns = self.nn_k_.kneighbors(X_class, return_distance=False)[:, 1:]
  File "C:\Users\iny2819\AppData\Local\anaconda3\lib\site-packages\sklearn\

test


In [50]:
len(dic_level1 )

37

In [21]:
syndigo_mapped.columns

Index(['GTIN', 'Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5',
       'Level 6', 'Level 7', 'Level 8', 'Level 9', 'Level 10', 'ITM_ID',
       'PMY_DPT_CD', 'PMY_DPT_DSC', 'REC_DPT_CD', 'REC_DPT_DSC', 'DPT_CD',
       'DPT_DSC', 'COM_CD', 'COM_DSC', 'SUBCOM_CD', 'SUBCOM_DSC',
       'VND_ECOM_DSC', 'ITEM_SUBCOM_text'],
      dtype='object')

In [46]:
 k_neighbors

3

In [45]:
subset_df['Level 2'].value_counts(ascending=True, dropna = False)

Absorbents                              3
Moisture / Mildew / Mold Removers       4
Sweepers                                6
Carpet & Floor Care                    19
Vacuums & Air Dusters                  21
Moisture & Mildew Absorbers            23
Vacuum Parts & Accessories             28
Product Dispensers                     45
Trash & Recycling                     384
Storage & Organization                407
Paper Products                        937
General Cleaning Tools                950
Air Fresheners                       1772
Cleaning Chemicals                   4483
Name: Level 2, dtype: int64

In [44]:
syndigo_mapped[syndigo_mapped['Level 1']==level_1]['Level 2'].value_counts(ascending=True,  dropna = False)

Adhesive Removers                       2
Absorbents                              3
Moisture / Mildew / Mold Removers       4
Sweepers                                6
Carpet & Floor Care                    19
Vacuums & Air Dusters                  21
Moisture & Mildew Absorbers            23
Vacuum Parts & Accessories             28
Product Dispensers                     45
Trash & Recycling                     384
Storage & Organization                407
Paper Products                        937
General Cleaning Tools                950
Air Fresheners                       1772
Cleaning Chemicals                   4483
Name: Level 2, dtype: int64

In [24]:

#syndigo_mapped['Level 1' ==  level_1]['Level 2'].value_counts(dropna = False) 
min_cnt = min(syndigo_mapped[syndigo_mapped['Level 1']==level_1]['Level 2'].value_counts(ascending=True))
k_neighbors = 5
if min_cnt < 3 :  k_neighbors  +  min_cnt  

2

In [17]:
level_1 = 'Health & Beauty'.

In [28]:
syndigo_mapped['Level 1' ==  level_1]['Level 2'].value_counts() 

KeyError: False

In [29]:
min(subset_df['Level 2'].value_counts(ascending=True))

2

In [30]:
series =  subset_df['Level 2'].value_counts(ascending=True, dropna = False)

In [31]:

if min(series) < 3:
    k_neighbors = 3
    subset_df = subset_df.drop(subset_df[subset_df['Level 2'].isin(series[series  < k_neighbors].index.tolist())].index) 

2

In [56]:
#type(X_train) # numpy.ndarray
len(X_train) #28

28

In [58]:


dic_level1= constrcut_dic()

dic_level1 = dict(list(dic_level1.items())[4: 5])

for level_1, filenamee in dic_level1.items():
    i += 1
    print(f'{i}. Process level 1 = {level_1}')
    x_subset_w2v, y_subset_w2v, subset_df, level2_id_map,  id2_level_map =  prepare_data_2(level_1)
    X_train, X_test, y_train, y_test = train_test_split(x_subset_w2v, y_subset_w2v.values, test_size= 0.2, stratify=y_subset_w2v, random_state=42)
    proc_predict(level_1)
    dic_y_train = dict(pd.Series(y_train).value_counts())
    smp_strgy = get_smp_strgy(dic_y_train, adj_ratio)
    proc_smote()


28. Process level 1 = Cleaning & Janitorial
Min cnt of Level 2 for class 1  Cleaning & Janitorial  is 2 and k_neighbors is 3
Training - w2v + MLP "Cleaning & Janitorial"
 Display MLP test  metrics
                                   precision    recall  f1-score   support

                   Air Fresheners       0.95      0.95      0.95       354
               Cleaning Chemicals       0.95      0.97      0.96       897
                   Paper Products       0.98      0.98      0.98       187
           General Cleaning Tools       0.93      0.85      0.88       190
           Storage & Organization       0.86      0.93      0.89        81
                Trash & Recycling       0.94      0.95      0.94        77
               Product Dispensers       0.67      0.44      0.53         9
              Carpet & Floor Care       0.00      0.00      0.00         4
      Moisture & Mildew Absorbers       0.75      0.60      0.67         5
       Vacuum Parts & Accessories       0.86      1.

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 2, n_neighbors = 4

In [65]:
unique, couints = np.unique(y_train, return_counts = True)
y_dic =dict(zip(unique, couints )) 
y_dic

{0: 1418,
 1: 3586,
 2: 750,
 3: 760,
 4: 326,
 5: 307,
 6: 36,
 7: 15,
 8: 18,
 9: 22,
 10: 5,
 11: 2,
 12: 17,
 13: 3}